In [5]:
from owslib.wms import WebMapService
import requests
from PIL import Image
from io import BytesIO
import rasterio
import resterio.plot

wms_url = "https://gis.toronto.ca/arcgis/services/basemap/cot_ortho/MapServer/WMSServer"
wms = WebMapService(wms_url)

bbox = (-79.393558267, 43.652245064, -79.390323394, 43.654593845)
width, height = 1024, 1024
layer = "0"

response = requests.get(wms_url, params={
    'service': 'WMS',
    'version': '1.1.1',
    'request': 'GetMap',
    'layers': layer,
    'styles': '',
    'bbox': ','.join(map(str, bbox)),
    'width': width,
    'height': height,
    'srs': 'EPSG:4326',
    'format': 'image/tiff'
})

with rasterio.open(BytesIO(response.content)) as src: 
    rasterio.plot.show(src)

print(src.crs)

ModuleNotFoundError: No module named 'resterio'